### Importing the reaktoro Python package

Using **Reaktoro** in Python requires first an import of the python package `reaktoro`:

In [1]:
from reaktoro import *

### Chemical system definition

The default thermodynamic databases embedded into Reaktoro is SUPCRT92, so you do not have to initialize the 
database `db = Database('supcrt98.xml')`, unless you use an alternative one.

Class `ChemicalEditor` provides convenient operations to initialize `ChemicalSystem` and `ReactionSystem` instances.

In [2]:
editor = ChemicalEditor()

Alternatively, the editor 

In [3]:
# Initialize a thermodynamic database with supcrt98.xml 
db = Database('supcrt98.xml')
#  Define the editor of the chemical system
editor = ChemicalEditor(db)

To define a chemical system, aqueous, gaseous, and mineral phases must be added. For aqueous phase, it can be done 
from a list of chemical element names. The database will be searched for all species that could be formed out of those 
elements. 

In [4]:
editor.addAqueousPhaseWithElements("H O C Ca Cl Mg")

To add gaseous phase, we provide a specific list of gaseous species that must be used:

In [5]:
editor.addGaseousPhase(["H2O(g)", "CO2(g)", "H2(g)", "O2(g)", "CH4(g)"])

For the `MineralPhase` object, we add two pure mineral phases:

In [6]:
editor.addMineralPhase("Calcite")
editor.addMineralPhase("Dolomite")

To create an object of class `ChemicalSystem` using the chemical system definition details stored in the object 
editor, we use

In [7]:
system = ChemicalSystem(editor)

To output the details of the chemical system to the console, i.e., the list of elements, species, and different 
phases, we can execute

In [8]:
print(system)

Aqueous                  Gaseous                  Calcite                  Dolomite                 
----------------------------------------------------------------------------------------------------
CO(aq)                   H2O(g)                   Calcite                  Dolomite                 
CO2(aq)                  CO2(g)                                                                     
CO3--                    H2(g)                                                                      
Ca(HCO3)+                O2(g)                                                                      
Ca++                     CH4(g)                                                                     
CaCO3(aq)                                                                                           
CaCl+                                                                                               
CaCl2(aq)                                                                                  

For the specific information about the chemical system, one can use the following methods:

In [9]:
print("Number of chemical species: ", len(system.species()))
print("Number of phases: ", len(system.phases()))
print("Number of elements: ", len(system.elements()))

print("List of species in chemical system: \n")
for species in system.species():
    print(species.name())

Number of chemical species:  37
Number of phases:  4
Number of elements:  7
List of species in chemical system: 

CO(aq)
CO2(aq)
CO3--
Ca(HCO3)+
Ca++
CaCO3(aq)
CaCl+
CaCl2(aq)
CaOH+
Cl-
ClO-
ClO2-
ClO3-
ClO4-
H+
H2(aq)
H2O(l)
H2O2(aq)
HCO3-
HCl(aq)
HClO(aq)
HClO2(aq)
HO2-
Mg(HCO3)+
Mg++
MgCO3(aq)
MgCl+
MgOH+
O2(aq)
OH-
H2O(g)
CO2(g)
H2(g)
O2(g)
CH4(g)
Calcite
Dolomite


### Defining the chemical equilibrium problem

Next, we create an equilibrium problem with our prescribed equilibrium conditions for
amounts of elements that are consistent with our intention of calculating reaction of calcite with 
injected 0.002 molal $\mathrm{MaCl_2}$ brine. Both temperature and pressure are assumed to be default values, i.e., 
25 $^\circ$C and 1 bar, respectively.

In [10]:
problem = EquilibriumProblem(system)
problem.add("H2O", 1, "kg")
problem.add("MgCl2", 0.002, "mol")
problem.add("CaCO3", 1, "mol")

### Calculating the chemical equilibrium state

In this step, we use the `equilibrate()` function to calculate the chemical equilibrium state of the system with the 
given equilibrium conditions stored in the object problem.

In [11]:
state = equilibrate(problem)

Reaktoro uses an efficient **Gibbs energy minimization** computation to determine the species amounts that correspond
 to a state of minimum Gibbs energy in the system, while satisfying the prescribed amount conditions for the 
 temperature, pressure, and element amounts. The result is stored in the object `state`, of class `ChemicalState`.

To output the result of equilibration to the console, we use

In [12]:
print(state)


Temperature [K]          Temperature [C]          Pressure [Pa]            Pressure [bar]           
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
298.15                   25                       100000                   1                        
Element                  Amount [mol]             Aqueous [mol]            Gaseous [mol]            Calcite [mol]            Dolomite [mol]           Dual Potential [kJ/mol]  
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
C                        1                        3.60918e-05              9.48936e-22              0.996215                 0.00374902               -91.0356            

Alternatively, to save equilibrated state into a file, one can use method `output()`:

In [13]:
state.output('result.txt')  # Output the equilibrium state into a file result.txt

To print the amounts of some specific aqueous species, one can use

In [14]:
# Print the amounts of some aqueous species
print('Amount of CO2(aq):', state.speciesAmount('CO2(aq)'))
print('Amount of HCO3-:', state.speciesAmount('HCO3-'))
print('Amount of CO3--:', state.speciesAmount('CO3--'))


Amount of CO2(aq): 2.280218470272726e-08
Amount of HCO3-: 2.5065301168597322e-05
Amount of CO3--: 3.3608545131281747e-06


Similarly, one can also print the amounts of certain element, say carbon, in both aqueous and gaseous phases

In [15]:
# Print the amounts of element C in both aqueous and gaseous phases
print('Amount of C in aqueous phase:', state.elementAmountInPhase('C', 'Aqueous'))
print('Amount of C in gaseous phase:', state.elementAmountInPhase('C', 'Gaseous'))
print('Amount of C in calcite phase:', state.elementAmountInPhase('C', 'Calcite'))

Amount of C in aqueous phase: 3.609182273072377e-05
Amount of C in gaseous phase: 9.489357152787924e-22
Amount of C in calcite phase: 0.9962148917732713
